In [2]:
import pandas as pd
from mwclient import Site
import json 
import requests


### Get split matches from a given team and split. Can return Game end and Game timeline

In [31]:
site = Site('lol.fandom.com', path="/")

tournament = "LCK/2024 Season/Spring Season"
team = 'Gen.G'

fields = [
    "PAB.Team1PicksByRoleOrder",
    "PAB.Team2PicksByRoleOrder",
    "MS.ShownName",
    "MSG.Blue",
    "MSG.Red",
    "MSG.Winner",
    "MSG.N_GameInMatch",
    "MS.BestOf",
    "PAB.Team1Ban1",
    *["PAB.Team{}Ban{}".format(i,j) for i in range(1,3) for j in range(1,6)],
    *["PAB.Team{}Pick{}".format(i,j) for i in range(1,3) for j in range(1,6)],
    "MS.DateTime_UTC",
    "MSG.RiotPlatformGameId",
   
]

cargo_query = site.api(
    "cargoquery",
    limit = "max",
    tables = "MatchSchedule = MS,MatchScheduleGame = MSG, PicksAndBansS7 = PAB",
    fields = ",".join(fields),
    where = 'MS.OverviewPage="{}"'.format(tournament) + 'OR MSG.Blue ="{}"'.format(team) +'OR MSG.Red ="{}"'.format(team),
    order_by = "MS.DateTime_UTC DESC",
    join_on = "MS.MatchId = MSG.MatchId, MSG.GameId = PAB.GameId"
)


APIError: ('cargoquery-invalidfieldalias', 'Error: Field alias "_pageName" starts with an underscore (_). This is not allowed in Cargo API queries.', 'See https://lol.fandom.com/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.')

In [32]:
df = pd.DataFrame([d['title'] for d in cargo_query['cargoquery']])
df_spring24 = df[df['ShownName'].isin(['LCK 2024 Spring','LCK 2024 Spring Playoffs'])]
df_spring24 = df_spring24[(df_spring24['Blue'] == team) | (df_spring24['Red'] == team)]
df_spring24 = df_spring24.rename(columns=lambda x: x.replace('Team1', 'Blue') if 'Team1' in x else x)
df_spring24 = df_spring24.rename(columns=lambda x: x.replace('Team2', 'Red') if 'Team2' in x else x)
df_spring24['RedPicksByRoleOrder'] = df_spring24['RedPicksByRoleOrder'].apply(lambda x: x.split(','))
df_spring24['BluePicksByRoleOrder'] = df_spring24['BluePicksByRoleOrder'].apply(lambda x: x.split(','))
df_spring24.reset_index(drop = True , inplace = True)
print(len(df_spring24))
df_spring24


53


,BluePicksByRoleOrder,RedPicksByRoleOrder,ShownName,Blue,Red,Winner,N GameInMatch,BestOf,BlueBan1,BlueBan2,...,BluePick4,BluePick5,RedPick1,RedPick2,RedPick3,RedPick4,RedPick5,DateTime UTC,RiotPlatformGameId,DateTime UTC__precision
0,"[Rumble, Viego, Corki, Kalista, Nautilus]","[Aatrox, Maokai, Azir, Draven, Poppy]",LCK 2024 Spring Playoffs,Gen.G,T1,1,1,5,Vi,Varus,...,Rumble,Viego,Azir,Maokai,Draven,Poppy,Aatrox,2024-04-14 06:00:00,LOLTMNT02_76144,0
1,"[K'Sante, Xin Zhao, Azir, Kalista, Neeko]","[Rek'Sai, Rell, Aurelion Sol, Varus, Annie]",LCK 2024 Spring Playoffs,T1,Gen.G,1,2,5,Nautilus,Twisted Fate,...,Xin Zhao,K'Sante,Varus,Rell,Aurelion Sol,Annie,Rek'Sai,2024-04-14 06:00:00,LOLTMNT02_76145,0
2,"[Rek'Sai, Sejuani, Azir, Aphelios, Lulu]","[Zac, Xin Zhao, Taliyah, Lucian, Nami]",LCK 2024 Spring Playoffs,Gen.G,T1,2,3,5,Vi,Varus,...,Sejuani,Rek'Sai,Lucian,Taliyah,Nami,Xin Zhao,Zac,2024-04-14 06:00:00,LOLTMNT02_76146,0
3,"[K'Sante, Kha'Zix, Aurelion Sol, Lucian, Nami]","[Gwen, Xin Zhao, Azir, Aphelios, Lulu]",LCK 2024 Spring Playoffs,Gen.G,T1,1,4,5,Vi,Varus,...,K'Sante,Kha'Zix,Lulu,Azir,Aphelios,Xin Zhao,Gwen,2024-04-14 06:00:00,LOLTMNT02_76147,0
4,"[Zac, Xin Zhao, Orianna, Lucian, Nami]","[K'Sante, Poppy, Corki, Zeri, Annie]",LCK 2024 Spring Playoffs,T1,Gen.G,2,5,5,Nautilus,Twisted Fate,...,Xin Zhao,Zac,Zeri,Annie,Corki,K'Sante,Poppy,2024-04-14 06:00:00,LOLTMNT02_76148,0
5,"[Urgot, Sejuani, Azir, Aphelios, Nautilus]","[Rek'Sai, Wukong, Akali, Zeri, Alistar]",LCK 2024 Spring Playoffs,Gen.G,Hanwha Life Esports,2,1,5,Maokai,Vi,...,Azir,Urgot,Zeri,Rek'Sai,Alistar,Wukong,Akali,2024-04-06 06:00:00,LOLTMNT02_70202,0
6,"[Aatrox, Sejuani, Aurelion Sol, Lucian, Nami]","[Jayce, Xin Zhao, Taliyah, Zeri, Lulu]",LCK 2024 Spring Playoffs,Gen.G,Hanwha Life Esports,1,2,5,Maokai,Vi,...,Aatrox,Aurelion Sol,Zeri,Xin Zhao,Lulu,Taliyah,Jayce,2024-04-06 06:00:00,LOLTMNT02_71173,0
7,"[Rek'Sai, Vi, Ahri, Jinx, Tahm Kench]","[Rumble, Jax, Azir, Zeri, Nautilus]",LCK 2024 Spring Playoffs,Hanwha Life Esports,Gen.G,2,3,5,Rell,Varus,...,Tahm Kench,Ahri,Zeri,Azir,Nautilus,Rumble,Jax,2024-04-06 06:00:00,LOLTMNT02_71199,0
8,"[Jayce, Vi, Taliyah, Zeri, Rakan]","[K'Sante, Lee Sin, Azir, Kai'Sa, Nautilus]",LCK 2024 Spring Playoffs,Hanwha Life Esports,Gen.G,2,4,5,Rell,Varus,...,Taliyah,Jayce,Nautilus,Azir,Kai'Sa,K'Sante,Lee Sin,2024-04-06 06:00:00,LOLTMNT02_71247,0
9,"[Twisted Fate, Vi, Azir, Senna, Tahm Kench]","[Rumble, Jax, Aurelion Sol, Kalista, Renata Gl...",LCK 2024 Spring Playoffs,Gen.G,Dplus KIA,1,1,5,Lee Sin,Rek'Sai,...,Vi,Azir,Kalista,Renata Glasc,Jax,Rumble,Aurelion Sol,2024-04-03 08:00:00,LOLTMNT02_69557,0


In [5]:

df_spring24.to_csv('df_spring24_geng_draft.csv')

In [33]:
#function that from the df_lec and a teamname gets blue and red side data as follow : 'Championplayed, pickOrder, pickOrder_2 (B1/B2/B3...) Position in game'
def get_red_blue_df(df,teamname):
    

    #Keeping only the data of the wanted Team and checking if both sides exists(here BDS)

    blue_none = False
    red_none = False
    #blue
    blue_df = df[df['Blue'] == teamname][['BluePicksByRoleOrder']]
    if blue_df.empty:
        print('No data for blue side')
        blue_none = True
        draft_blue = pd.DataFrame()
    #red  
    red_df = df[df['Red'] == teamname][['RedPicksByRoleOrder']]
    if red_df.empty: 
        print('No data for red side')
        red_none = True
        draft_red = pd.DataFrame()
    #both  
    if red_none and blue_none:
        print('No data for blue AND red side')
        return None,None
    
    #mapping for position and pickorder_2 column
    position_mapping = {1: 'TOP', 2: 'JGL', 3: 'MID', 4: 'ADC', 5: 'SUPP'}
    pickorder_blue_mapping = {1: 'B1', 2: 'B2/B3', 3: 'B2/B3', 4: 'B4/B5', 5: 'B4/B5'}
    pickorder_red_mapping = {1: 'R1/R2', 2: 'R1/R2', 3: 'R3', 4: 'R4', 5: 'R5'}
    
    #Creating Red df if exists :
    if not red_df.empty:
        new_data = []
        for index, row in red_df.iterrows():
            print(row['RedPicksByRoleOrder'].items())
            for champion, order in row['RedPicksByRoleOrder'].items():  # Iterate through each champion and its order in the draft

                index_of_key = list(row['RedPicksByRoleOrder'].keys()).index(champion) # Get the index of the key in the original dictionary
                new_data.append({'champion': champion, 'pickOrder': order, 'position': index_of_key +1 }) # Append the data to the list
        
        draft_red = pd.DataFrame(new_data)
        print(draft_red) # Create a new DataFrame from the list of data
        draft_red['position'] = draft_red['position'].map(position_mapping)
        draft_red['Red Side Picks'] = draft_red['pickOrder'].map(pickorder_red_mapping)
    #----------------------------------
    
    #Creating Blue df if exists : 
    if not blue_df.empty:
        new_data = []
        for index, row in blue_df.iterrows():
            
            for champion, order in row['BluePicksByRoleOrder'].items():
                
                index_of_key = list(row['BluePicksByRoleOrder'].keys()).index(champion)
                new_data.append({'champion': champion, 'pickOrder': order, 'position': index_of_key +1 })
                
        draft_blue = pd.DataFrame(new_data)
        draft_blue['position'] = draft_blue['position'].map(position_mapping)
        draft_blue['Blue Side Picks'] = draft_blue['pickOrder'].map(pickorder_blue_mapping)


        
    
        
    return draft_red,draft_blue

def get_prio_position_draft(df,team_name):

    #get the list as dict (pickbyroleorder)
    for index, row in df.iterrows():
        blue_champions_dict = {}
        red_champions_dict = {}
        # Iterate over the champions in the "BluePicksByRoleOrder" column
        for champion in row['BluePicksByRoleOrder']:

            for column_index, value in enumerate(row):

                if champion == df.iloc[index,column_index]:
                    blue_champions_dict[champion] = column_index - 17
                
        for champion in row['RedPicksByRoleOrder']:

            for column_index, value in enumerate(row):
                        
                if champion == df.iloc[index,column_index]:
                    red_champions_dict[champion] = column_index - 22

        df.at[index, 'BluePicksByRoleOrder'] = blue_champions_dict
        df.at[index, 'RedPicksByRoleOrder'] = red_champions_dict  

    df_red,df_blue = get_red_blue_df(df,team_name)  

    
    desired_order = ['TOP', 'JGL', 'MID', 'ADC', 'SUPP']
    most_picked_champions = df_blue['champion'].value_counts()
    position_b1_pick = df_blue[df_blue['pickOrder'] == 1]['position'].value_counts()
    position_r3_pick = df_red[df_red['pickOrder'] == 3]['champion'].value_counts()
    position_r5_pick = df_red[df_red['pickOrder'] == 5]['champion'].value_counts()


    # Set the position column as categorical with the desired order
    df_red['position'] = pd.Categorical(df_red['position'], categories=desired_order, ordered=True)
    df_blue['position'] = pd.Categorical(df_blue['position'], categories=desired_order, ordered=True)

    # Pivot the DataFrame and calculate the percentage of occurrences for each combination
    matrix_counts_blue = df_blue.pivot_table(index='position', columns='Blue Side Picks', aggfunc='size', fill_value=0, observed = True)
    matrix_percentages_blue = (matrix_counts_blue.div(matrix_counts_blue.sum(axis=1), axis=0) * 100).round(1)

    matrix_counts_red = df_red.pivot_table(index='position', columns='Red Side Picks', aggfunc='size', fill_value=0, observed = True)
    matrix_percentages_red = (matrix_counts_red.div(matrix_counts_red.sum(axis=1), axis=0) * 100).round(2)

    return most_picked_champions, position_b1_pick, position_r3_pick, position_r5_pick, matrix_counts_blue, matrix_counts_red, matrix_percentages_blue, matrix_percentages_red

In [35]:
a,b,c,d,e,f,g,h = get_prio_position_draft(df_spring24,'Gen.G')

dict_items([("Rek'Sai", 5), ('Rell', 2), ('Aurelion Sol', 3), ('Varus', 1), ('Annie', 4)])
dict_items([("K'Sante", 4), ('Poppy', 5), ('Corki', 3), ('Zeri', 1), ('Annie', 2)])
dict_items([('Rumble', 4), ('Jax', 5), ('Azir', 2), ('Zeri', 1), ('Nautilus', 3)])
dict_items([("K'Sante", 4), ('Lee Sin', 5), ('Azir', 2), ("Kai'Sa", 3), ('Nautilus', 1)])
dict_items([('Jayce', 4), ('Jax', 2), ('Tristana', 5), ('Draven', 3), ('Rumble', 1)])
dict_items([('Urgot', 5), ('Maokai', 2), ('Azir', 4), ('Zeri', 1), ('Lulu', 3)])
dict_items([("K'Sante", 4), ('Rell', 2), ('Aurelion Sol', 5), ('Varus', 1), ('Hwei', 3)])
dict_items([('Twisted Fate', 4), ('Jax', 1), ('Yone', 5), ('Zeri', 2), ('Lulu', 3)])
dict_items([("K'Sante", 5), ('Rell', 3), ('Orianna', 4), ('Lucian', 1), ('Nami', 2)])
dict_items([('Renekton', 4), ('Maokai', 2), ('Corki', 5), ('Lucian', 1), ('Milio', 3)])
dict_items([('Udyr', 5), ('Sejuani', 3), ('Tristana', 4), ('Lucian', 1), ('Nami', 2)])
dict_items([("K'Sante", 4), ('Rell', 2), ('Corki'

In [37]:
f

Red Side Picks,R1/R2,R3,R4,R5
position,,,,
TOP,0,3,12,13
JGL,9,7,4,8
MID,4,7,10,7
ADC,24,3,1,0
SUPP,19,8,1,0


In [28]:
list_games_bug = ['LOLTMNT02_58682',
'LOLTMNT02_59611',
'LOLTMNT02_55608',
'LOLTMNT02_55648',
'LOLTMNT02_53568',
'LOLTMNT02_53589',
'LOLTMNT02_58454',
'LOLTMNT02_58466',
'LOLTMNT02_55642',
'LOLTMNT02_56440',
'LOLTMNT02_41281',
'LOLTMNT02_42276',
'LOLTMNT02_41039',
'LOLTMNT02_41040',
'LOLTMNT02_31940',
'LOLTMNT02_32961',
'LOLTMNT02_31982',
'LOLTMNT02_29804',
'LOLTMNT02_30776',
'LOLTMNT02_25093',
'LOLTMNT02_25120',
'LOLTMNT02_22225',
'LOLTMNT02_21273',
'LOLTMNT02_21295']

df_bug = df_spring24[df_spring24['RiotPlatformGameId'].isin(list_games_bug)]


# Import des games data end & timeline avec les matchsIds

In [29]:
# get the game data from leaguepedia (game_end, timeline) using the RiotPlatformGameId as input
def game_format(id_game):
    gameDatas = json.loads(requests.get(f"https://lol.fandom.com/api.php?action=query&format=json&prop=revisions&titles=V5%20data%3A{id_game}&rvprop=content&rvslots=main").content)
    gameDatas = json.dumps(gameDatas)
    gameDatas = gameDatas.replace(' ', '')
    gameDatas = gameDatas.replace('\\n', '')
    gameDatas = json.loads(gameDatas)
    for y in gameDatas['query']['pages']:
        gameDatas = gameDatas['query']['pages'][y]['revisions'][0]['slots']['main']['*']
    gameDatas = json.loads(gameDatas)

    gameTimeline = json.loads(requests.get(f"https://lol.fandom.com/api.php?action=query&format=json&prop=revisions&titles=V5%20data%3A{id_game}%2FTimeline&rvprop=content&rvslots=main").content)
    gameTimeline = json.dumps(gameTimeline)
    gameTimeline = gameTimeline.replace(' ', '')
    gameTimeline = gameTimeline.replace('\\n', '')
    gameTimeline = json.loads(gameTimeline)
    for y in gameTimeline['query']['pages']:
        gameTimeline = gameTimeline['query']['pages'][y]['revisions'][0]['slots']['main']['*']
    gameTimeline = json.loads(gameTimeline)

    return gameDatas,gameTimeline

In [30]:
list_game_end = []
list_game_timeline = []
for game_id in df_spring24['RiotPlatformGameId'].unique().tolist():
    game_end,game_timeline = game_format(game_id)
    list_game_end.append(game_end)
    list_game_timeline.append(game_timeline)


In [10]:
with open('list_game_timeline_geng_compet.json', "w") as file:
    json.dump(list_game_timeline, file)
with open('list_game_end_geng_compet.json', "w") as file:
    json.dump(list_game_end, file)

In [ ]:



def get_df_event(list_game_timeline_geng_compet):

    # condition :  game['frames'][i]['events'][j]['type'] == 'CHAMPION_KILL'
    # position : game['frames'][i]['events'][j]['position']['x'] & game['frames'][i]['events'][j]['position']['y']
    # killer : game['frames'][i]['events'][j]['killerId'] /// list participants ids assists : game['frames'][i]['events'][j]['assistingParticipantIds'] 
    # deaths : game['frames'][i]['events'][j]['victimId']
    # timestamp : game['frames'][i]['events'][j]['timestamp']


    list_dict_events = []

    for game in list_game_timeline_geng_compet:
        
        participants = game['participants']

        try:
            id_to_puuid = {participant['participantId']: participant['puuid'] for participant in participants}

            def to_puuid(participant_id):
                # Swap participant IDs to PUUIDs
                return id_to_puuid.get(participant_id)
            
            def to_puuid_lists(participant_ids):
                # Swap participant IDs to PUUIDs
                puuids = [id_to_puuid[participant_id] for participant_id in participant_ids]
                return puuids

            
            for i in range(len(game['frames'])):

                for j in range(len(game['frames'][i]['events'])):
                    dict_events = {}
                    if game['frames'][i]['events'][j]['type'] == 'CHAMPION_KILL':
                        
                        dict_events['game_id'] = game['gameId']
                        dict_events['event_type'] = game['frames'][i]['events'][j]['type']
                        dict_events['killer'] = to_puuid(game['frames'][i]['events'][j]['killerId'])
                        dict_events['deaths'] = to_puuid(game['frames'][i]['events'][j]['victimId'])
                        try:
                            dict_events['assists'] = to_puuid_lists(game['frames'][i]['events'][j]['assistingParticipantIds'])
                        except KeyError:
                            dict_events['assists'] = None
                        dict_events['x'] = game['frames'][i]['events'][j]['position']['x']
                        dict_events['y'] = game['frames'][i]['events'][j]['position']['y']
                        dict_events['timestamp'] = game['frames'][i]['events'][j]['timestamp'] / 60000
                        list_dict_events.append(dict_events)     

        except KeyError:
            print(game['gameId'])

    return pd.DataFrame(list_dict_events)

df_events = get_df_event(list_game_timeline_geng_compet)

